### Inverse Kinematics of Franka Emika `Panda`

In [1]:
import numpy as np
np.set_printoptions(precision=2)
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from mujoco_parser import MuJoCoParserClass
print ("Done.")

Done.


### Move a body of `Panda` while maintaining the orientation

In [2]:
env = MuJoCoParserClass(name='PAPRAS',rel_xml_path='../asset/papras/papras.urdf.xml',VERBOSE=False)
env.init_viewer(TERMINATE_GLFW=True,INITIALIZE_GLFW=True,window_width=0.5,window_height=0.5)
env.set_max_tick(max_tick=100)

# Prepare for IK
body_name = 'robot1/end_link' # panda_eef / panda_link_4 / panda_link_5 / panda_link_6
q         = env.get_q_pos(q_pos_idxs=env.rev_joint_idxs)
p_EE      = env.get_p_body(body_name=body_name)
R_EE      = env.get_R_body(body_name=body_name)
p_trgt    = p_EE + np.array([0.5,0.0,-0.5])
R_trgt    = R_EE

# Buffers
err_list = np.zeros(env.max_tick)
q_list   = np.zeros((env.max_tick,env.n_rev_joint))
while env.IS_ALIVE():
    # Numerical IK
    dq,err = env.one_step_ik(body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,th=3.0*np.pi/180.0)
    q = q + dq
    # FK
    env.forward(q_pos=q,q_pos_idxs=env.rev_joint_idxs)
    env.add_marker(env.get_p_body(body_name),radius=0.1,color=np.array([1,0,0,0.5]))
    env.add_marker(p_trgt,radius=0.1,color=np.array([0,0,1,0.5])) 
    env.render(RENDER_ALWAYS=True)
    # Append
    q_list[env.tick-1,:] = env.get_q_pos(q_pos_idxs=env.rev_joint_idxs)
print ("Done.")

Creating window glfw
Creating window glfw
Done.


: 

In [ ]:
env.init_viewer(TERMINATE_GLFW=True,INITIALIZE_GLFW=True)
fig = plt.figure(figsize=(15,6))
for idx,tick in enumerate(np.linspace(0,env.max_tick-1,8).astype(np.int32)):
    env.forward(q_pos=q_list[tick,:],q_pos_idxs=env.rev_joint_idxs)
    img = env.plot_scene(render_w=400,render_h=300,
                         cam_distance=2.0,cam_elevation=-45,cam_lookat=[0,0,1.25],RETURN_IMG=True)
    ax = fig.add_subplot(2,4,idx+1)
    ax.imshow(img)
    ax.set_title('[%d/%d]'%(tick,env.max_tick-1),fontsize=12)